# Data Mining Lab 2


In [7]:
import scipy.io
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load the .mat files
mat_invol = scipy.io.loadmat('epochs_blinks_ASR_invol.mat')
mat_vol = scipy.io.loadmat('epochs_blinks_ASR_vol.mat')

# Extract the data from the .mat files
data_invol = mat_invol['epochs_blinks_ASR_invol']
data_vol = mat_vol['epochs_blinks_ASR_vol']

# Number of features ('FP1' and 'FP2'), time steps and samples
n_features = data_invol.shape[0]
n_timesteps = data_invol.shape[1]
n_samples_invol = data_invol.shape[2]
n_samples_vol = data_vol.shape[2]

# Reshape the data to be of size (samples, timesteps, features)
data_invol = np.transpose(data_invol, (2, 1, 0))
data_vol = np.transpose(data_vol, (2, 1, 0))

# Create labels for the involuntary (0) and voluntary (1) data
labels_invol = np.zeros((n_samples_invol, 1))
labels_vol = np.ones((n_samples_vol, 1))

# Combine the involuntary and voluntary data and labels
X = np.concatenate((data_invol, data_vol), axis=0)
y = np.concatenate((labels_invol, labels_vol), axis=0)


In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, Bidirectional
from keras.layers import MaxPooling1D, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

# Split the data into training and temporary sets (70% training, 30% temporary)
X_temp, X_train, y_temp, y_train = train_test_split(X, y, test_size=0.7, random_state=42, shuffle=True)

# Split the temporary set into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)


In [20]:
from keras.optimizers import Adam
# Define the model
model = Sequential()

# Convolutional layer
model.add(Conv1D(32, 5, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(1))

# Batch normalization layer
model.add(BatchNormalization())

# LSTM layer
model.add(Bidirectional(LSTM(25, return_sequences=True)))  # Bidirectional LSTM layer with 50 units

# Dropout for regularization
model.add(Dropout(0.3))

# LSTM layer
model.add(Bidirectional(LSTM(50)))  # Another Bidirectional LSTM layer with 50 units

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))  

learning_rate = 0.002  # Set the learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Test the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

Epoch 1/10
20/20 [==============================] - 34s 2s/step - loss: 0.6338 - accuracy: 0.6378 - val_loss: 0.8603 - val_accuracy: 0.5662
Epoch 2/10
20/20 [==============================] - 33s 2s/step - loss: 0.5380 - accuracy: 0.7291 - val_loss: 0.7397 - val_accuracy: 0.6029
Epoch 3/10
20/20 [==============================] - 34s 2s/step - loss: 0.5061 - accuracy: 0.7780 - val_loss: 0.6577 - val_accuracy: 0.5956
Epoch 4/10
20/20 [==============================] - 34s 2s/step - loss: 0.4757 - accuracy: 0.7811 - val_loss: 0.7083 - val_accuracy: 0.6029
Epoch 5/10
20/20 [==============================] - 33s 2s/step - loss: 0.4586 - accuracy: 0.7969 - val_loss: 0.6502 - val_accuracy: 0.6324
Epoch 6/10
20/20 [==============================] - 34s 2s/step - loss: 0.4314 - accuracy: 0.8063 - val_loss: 0.5925 - val_accuracy: 0.6912
Epoch 7/10
20/20 [==============================] - 34s 2s/step - loss: 0.4076 - accuracy: 0.8205 - val_loss: 0.5704 - val_accuracy: 0.6765
Epoch 8/10
20/20 [==

In [21]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import argmax

# predict the values from the test dataset
y_pred = model.predict(X_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]  # transform probabilities into binary outputs

# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap="Blues")
plt.title('Confusion matrix')
plt.xlabel('Predicted Voluntary Blinking')
plt.ylabel('True Voluntary Blinking')
plt.show()

5/5 [==============================] - 3s 600ms/step - loss: 0.7283 - accuracy: 0.6618
Test loss: 0.7282834649085999, Test accuracy: 0.6617646813392639


In [6]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_test, y_pred)

print(report)

Shape of Training df:  (3613, 4)
Shape of Testing df:  (347, 4)


In [ ]:
import numpy as np
from scipy import signal

def calculate_psd(data, Fs):
    """
    Calculate the power spectral density (PSD) for EEG data.

    Parameters:
    - data: array-like, EEG data with shape (channels, points, trials)
    - Fs: int, the sampling frequency

    Returns:
    - psds: array-like, PSDs with shape (channels, frequencies, trials)
    - freqs: array-like, frequencies for which the PSDs were computed
    """
    # Initialize array to store the PSDs
    psds = []

    # Loop over the channels
    for channel_data in data:
        # Loop over the trials
        channel_psds = []
        for trial_data in channel_data.T:
            # Calculate the PSD using Welch's method
            freqs, psd = signal.welch(trial_data, Fs)
            channel_psds.append(psd)
        psds.append(channel_psds)
    
    return np.array(psds), freqs

In [ ]:
# Extract the data from the .mat files
data_invol = mat_invol['epochs_blinks_ASR_invol']
data_vol = mat_vol['epochs_blinks_ASR_vol']
​
Fs=250; #sample frequency
# Calculate the PSDs of the GNAN data
psds_invol, freqs = calculate_psd(data_invol, Fs)
psds_vol, _ = calculate_psd(data_vol, Fs)
​
# Number of features ('FP1' and 'FP2'), time steps and samples
n_features = psds_invol.shape[0]
n_timesteps = psds_invol.shape[2]
n_samples_invol = psds_invol.shape[1]
n_samples_vol = psds_vol.shape[1]
​
# Reshape the data to be of size (samples, timesteps, features)
psds_invol = np.transpose(psds_invol, (1, 2, 0))
psds_vol = np.transpose(psds_vol, (1, 2, 0))
​
# Create labels for the involuntary (0) and voluntary (1) data
labels_invol = np.zeros((n_samples_invol, 1))
labels_vol = np.ones((n_samples_vol, 1))
​
# Combine the involuntary and voluntary data and labels
X = np.concatenate((psds_invol, psds_vol), axis=0)
y = np.concatenate((labels_invol, labels_vol), axis=0)

In [ ]:
# Split the data into training and temporary sets (70% training, 30% temporary)
X_temp, X_train, y_temp, y_train = train_test_split(X, y, test_size=0.7, random_state=42, shuffle=True)

# Split the temporary set into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

In [ ]:
# Define the model
model = Sequential()

# Convolutional layer
model.add(Conv1D(128, 3, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(3))

# Batch normalization layer
model.add(BatchNormalization())

# LSTM layer
model.add(Bidirectional(LSTM(25, return_sequences=True)))  # Bidirectional LSTM layer with 50 units

# Dropout for regularization
model.add(Dropout(0.5))

# LSTM layer
model.add(Bidirectional(LSTM(75)))  # Another Bidirectional LSTM layer with 50 units

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))  

learning_rate = 0.003  # Set the learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))

# Test the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

In [ ]:
# predict the values from the test dataset
y_pred = model.predict(X_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]  # transform probabilities into binary outputs

# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap="Blues")
plt.title('Confusion matrix')
plt.xlabel('Predicted Voluntary Blinking')
plt.ylabel('True Voluntary Blinking')
plt.show()

In [ ]:
# Generate the classification report
report = classification_report(y_test, y_pred)

print(report)

In [ ]:
# Load the .mat files
mat_invol_jap = scipy.io.loadmat('/kaggle/input/jap-epoch-raw-data/jap_blink_involuntary.mat')
mat_vol_jap = scipy.io.loadmat('/kaggle/input/jap-epoch-raw-data/jap_blink_voluntary.mat')

# Extract the data from the .mat files
data_invol_jap = mat_invol_jap['jap_blink_involuntary']
data_vol_jap = mat_vol_jap['jap_blink_voluntary']

# Number of features ('FP1' and 'FP2'), time steps and samples
n_features_jap = data_invol_jap.shape[0]
n_timesteps_jap = data_invol_jap.shape[1]
n_samples_invol_jap = data_invol_jap.shape[2]
n_samples_vol_jap = data_vol_jap.shape[2]

# Reshape the data to be of size (samples, timesteps, features)
data_invol_jap = np.transpose(data_invol_jap, (2, 1, 0))
data_vol_jap = np.transpose(data_vol_jap, (2, 1, 0))

# Create labels for the involuntary (0) and voluntary (1) data
labels_invol_jap = np.zeros((n_samples_invol_jap, 1))
labels_vol_jap = np.ones((n_samples_vol_jap, 1))

# Combine the involuntary and voluntary data and labels
X_jap = np.concatenate((data_invol_jap, data_vol_jap), axis=0)
y_jap = np.concatenate((labels_invol_jap, labels_vol_jap), axis=0)

In [ ]:
# Split the data into training and temporary sets (70% training, 30% temporary)
X_temp_jap, X_train_jap, y_temp_jap, y_train_jap = train_test_split(X_jap, y_jap, test_size=0.7, random_state=42, shuffle=True)

# Split the temporary set into validation and test sets (50% validation, 50% test)
X_val_jap, X_test_jap, y_val_jap, y_test_jap = train_test_split(X_temp_jap, y_temp_jap, test_size=0.5, random_state=42, shuffle=True)

In [ ]:
# Define the model
model_jap = Sequential()

# Convolutional layer
model_jap.add(Conv1D(64, 3, activation='relu', input_shape=(n_timesteps_jap, n_features_jap)))
model_jap.add(MaxPooling1D(3))

# Batch normalization layer
model_jap.add(BatchNormalization())

# LSTM layer
model_jap.add(Bidirectional(LSTM(25, return_sequences=True)))  # Bidirectional LSTM layer with 50 units

# Dropout for regularization
model_jap.add(Dropout(0.5))

# LSTM layer
model_jap.add(Bidirectional(LSTM(50)))  # Another Bidirectional LSTM layer with 50 units

# Output layer for binary classification
model_jap.add(Dense(1, activation='sigmoid'))  

learning_rate = 0.00021  # Set the learning rate
optimizer_jap = Adam(learning_rate=learning_rate)
model_jap.compile(loss='binary_crossentropy', optimizer=optimizer_jap, metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
history_jap = model_jap.fit(X_train_jap, y_train_jap, epochs=100, batch_size=32, validation_data=(X_val_jap, y_val_jap))

# Test the model on the test data
test_loss_jap, test_accuracy_jap = model_jap.evaluate(X_test_jap, y_test_jap)
print(f'Test loss: {test_loss_jap}, Test accuracy: {test_accuracy_jap}')

In [ ]:
# predict the values from the test dataset
y_pred_jap = model_jap.predict(X_test_jap)
y_pred_jap = [1 if y_jap >= 0.5 else 0 for y_jap in y_pred_jap]  # transform probabilities into binary outputs

# Compute the confusion matrix
conf_mat_jap = confusion_matrix(y_test_jap, y_pred_jap)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_mat_jap, annot=True, fmt='d', cmap="Blues")
plt.title('Confusion matrix')
plt.xlabel('Predicted Voluntary Blinking')
plt.ylabel('True Voluntary Blinking')
plt.show()

In [ ]:
# Generate the classification report
report_jap = classification_report(y_test_jap, y_pred_jap)

print(report_jap)